# Simple bot

In [9]:
import string
import nltk
import annoy
import numpy as np
import tensorflow as tf
import tqdm
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
# from tqdm import tqdm_notebook
from transformers import TFAutoModel, AutoTokenizer

In [16]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("../data/Otvety.txt", "r") as fin:
        for line in tqdm.notebook.tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [17]:
!head -3 prepared_answers.txt

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 
Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. 


In [18]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [19]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [20]:
bert = TFAutoModel.from_pretrained("setu4993/LaBSE")
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

2021-10-23 22:00:43.918371: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-23 22:00:43.920897: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-23 22:00:43.925582: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without fur

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [22]:
q_len = []
a_len = []
with open("prepared_answers.txt", "r") as f:
    counter = 0
    for line in tqdm.notebook.tqdm(f):
        spls = line.split("\t")
        q_len.append(len(spls[0].split()))
        a_len.append(len(spls[1].split()))

0it [00:00, ?it/s]

In [23]:
np.median(q_len), np.mean(q_len), np.quantile(q_len, 0.75), np.quantile(q_len, 0.85), np.quantile(q_len, 0.95), np.max(q_len)

(12.0, 20.454707214215595, 21.0, 31.0, 63.0, 862)

In [24]:
bert_index = annoy.AnnoyIndex(768 , 'angular')

index_map = {}
counter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm.notebook.tqdm(f):
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
        
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
        out_state = bert(**tok)[1].numpy()[0]
        bert_index.add_item(counter, out_state)
        counter += 1
        

bert_index.build(10)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
bert_index.save('bert_index')

In [1]:
bert_index.build(10)

NameError: name 'bert_index' is not defined

In [24]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers = index.get_nns_by_vector(vector, 2)
    return [index_map[i] for i in answers]

In [18]:
get_response("какой навигатор выбрать?", bert_index, index_map)

['В семью или на работу?. \n']

In [19]:
get_response("какие новые фильмы есть?", bert_index, index_map)

['99 франков (99 francs) <br>Комедия (Франция, 2008г. )<br>Режиссер: Жан Коунен<br>В ролях: Жан Дюжарден<br>Жослин Киврин<br>Патрик Милле<br><br>Апостол (Апостол) <br>Драма, Военный, Сериал (Россия, 2008г. )<br>Режиссер: Юрий Мороз<br>В ролях: Евгений Миронов<br>Николай Фоменко<br>Дарья Мороз<br><br>Астерикс на Олимпийских играх (Asterix aux jeux olympiques) <br>Комедия, Приключения, Семейный (Франция, Германия, Испания, 2008г. )<br>Режиссер: Томас Лангманн<br>В ролях: Жерар Депардье<br>Кловис Корнилляк<br>Ален Делон<br><br>Бабка Ёжка и другие (Бабка Ёжка и другие) <br>Мультфильм (Россия, 2008г. )<br>Режиссер: Валерий Угаров<br>В ролях: Армен Джигарханян<br>Татьяна Бондаренко<br>Алексей Колган<br><br>Вне кольца (Beyond the Ring) <br>Боевик, Драма (США, 2008г. )<br>Режиссер: Герсон Сангинитто<br>В ролях: Андре Лима<br>Мартин Коув<br>Гари Бьюзи<br><br>Вопреки здравому смыслу (Опровергая Эвклида) <br>Мелодрама (Россия, 2008г. )<br>Режиссер: Сайдо Курбанов<br>В ролях: Ирина Апексимова<br>Э

In [21]:
get_response("какие новые фильмы посмотреть?", bert_index, index_map)

['Отпуск по обмену / The Holiday (2006 <br>Идеальное Рождество (Снежный глобус) / Snowglobe (2007) <br>Рождественский коттедж / Christmas Cottage (2008) <br>Ёлки (2010) <br>Новогодние сваты (2010) <br>Снежный принц / Snow Prince (2009) <br>Суженый-ряженый (2007) <br>Полночный поцелуй" (2008) <br>Ирония судьбы 2: Продолжение (2007) <br>История любви, или Новогодний розыгрыш (2008) <br>Карнавальная ночь-2, или 50 лет спустя (2006) <br>Новогодние сваты (2010) <br>Удивительная история 2008 <br>Идеальное Рождество 2007 <br>Реальная любовь 2003 <br>Эван Всемогущий 2007 <br>Четыре Рождества 2008 <br>Рождественский коттедж 2008 <br>Санта на продажу 2010 <br>Санта-Хрякус: Страшдественская сказка 2006 <br>Свадьба на Рождество 2006 <br>Нянька на Рождество 2010 <br>Добро пожаловать, или Соседям вход 2006 <br>Миллион на Рождество 2007 <br>Наконец-то Рождество 2008 <br>Наследство к Рождеству 2007 <br>Новогодняя засада (2008) <br>Новогодняя семейка (2008) <br>Новогодние приключения в июле (2008) <br>

In [25]:
get_response("куда можно сходить?", bert_index, index_map)

['Это конкретное предложение? или просто так ...интересуешься?. \n',
 'Только не под себя.... \n']